In [67]:
import numpy as np
import scipy.sparse as sparse
import random
import hashlib

# Load in Data

In [68]:
data = np.load('Data/user_movie_rating.npy')[:,0:2]

In [69]:
col = data[:, 0]
row = data[:, 1]

n_col = col.max() + 1
n_row = row.max() + 1 

M = sparse.coo_matrix((np.ones(len(data)), (row, col)), shape=(n_row, n_col)).tocsr()
small_M  = M[:,0:1000]

In [70]:
perm = 100 
perm_seeds = []
row_len,_ = small_M.get_shape()
seed = 1000
for i in range(perm):
    perm_seeds.append(int(hashlib.sha256(str(seed+i).encode()).hexdigest(),16)%4294967295)


In [71]:
from tqdm import tqdm


def minhashing(M):
    signature = []
    index_perm = np.arange(M.shape[0])#reshuffeling the same index movies_uay
    sig_len = M.shape[1]-1
    for seed in tqdm(perm_seeds):
        random.seed(seed)
        np.random.shuffle(index_perm)
        perm_M = M[index_perm[:1000],1:] #Just take the top 1000 because it is very likely for each user to be in the first thousand movies at least once
        non_zero = perm_M.nonzero()
        sig = non_zero[0][np.unique(non_zero[1], return_index=True)[1]]
        if len(sig)==sig_len:
            signature.append(sig)        
    return signature
        

        


In [72]:
a = minhashing(M)

100%|██████████| 100/100 [00:30<00:00,  3.29it/s]


In [73]:
u = np.array(a).T.tolist()

In [74]:
len(u)

103703

In [75]:
len(u[0])

100

In [76]:
print(u[1244][0:9])

[11, 54, 11, 4, 54, 7, 30, 29, 0]


In [77]:
print(u[3086][0:9])

[11, 12, 11, 21, 18, 12, 31, 29, 0]


In [87]:
def LSH(M,b,n_rows):
    #Create Bands and Hashes
    n_users = len(M)
    sign_len = len(M[0])   
    buckets = [ ]
    candidates = []
    for band in tqdm(range(b)):
        bucket = {}
        candidate_in_band = []
        if n_rows*(band + 1) - 1 > sign_len:
                break
        for i in range(n_users):
            
            hash_num = (int((hashlib.sha256(f"{u[i][n_rows*band:(n_rows*(band + 1) - 1)]}".encode())).hexdigest(), 16))
            if  hash_num not in bucket:
                bucket[hash_num] = []
            bucket[hash_num].append(i)
            if len(bucket[hash_num])==2:
                candidate_in_band.append(hash_num)
        buckets.append(bucket)
        candidates.append(candidate_in_band)
    return buckets, candidates     
# implement hash table to check for identity in linear time

        


In [109]:
#
resultado, candidates = LSH(u,10,10)

100%|██████████| 12/12 [00:03<00:00,  3.48it/s]


In [110]:
len(candidates[0])

5245

In [111]:
candidates[0][11]

20027077779057714479364797238981222013171616776089078644952826234833096565794

In [112]:
candidates[5][0]

26324522487920728352798412458193921085038097059818490949900006547126486230678

In [113]:
resultado[0][20027077779057714479364797238981222013171616776089078644952826234833096565794]

[701, 2035]

In [114]:
MT = M.T

In [115]:
def Jaccard_similarity(pair,M):
    u1 = int(pair[0])
    u2 = int(pair[1])
    movies_u1 = M[u1]
    movies_u2 = M[u2]
    intersection = movies_u1.dot(movies_u2.T).sum()
    union = (movies_u1 + movies_u2).count_nonzero()
    similarity = intersection/union
    return float(similarity)

In [116]:
def get_pairs(candidates, buckets,M):
    pairs = {}
    for i in tqdm(range(len(candidates))):
        for j in range(len(candidates[i])):
            #Added this limit because we were getting too many pairs, I think it's too low but we won't be able to execute the Jaccard in time with the current amount of pairs
            if len(buckets[i][candidates[i][j]]) <= 2:
                for z in range(len(buckets[i][candidates[i][j]]) - 1):
                    for p in range(z + 1, len(buckets[i][candidates[i][j]])):
                        u1 = buckets[i][candidates[i][j]][z]
                        u2 = buckets[i][candidates[i][j]][p]
                        if f"{u1},{u2}" not in pairs and f"{u2},{u1}" not in pairs:
                        #    similarity = Jaccard_similarity(u1,u2,MT)
                        #    if similarity > 0.5:
                            pairs[f"{u1},{u2}"] = 1
                        elif f"{u1},{u2}" in pairs:
                            pairs[f"{u1},{u2}"] = pairs[f"{u1},{u2}"] + 1
    return pairs   

In [117]:
pairs = get_pairs(candidates, resultado, MT)
len(pairs)

100%|██████████| 12/12 [00:00<00:00, 137.69it/s]


38843

In [107]:
#Not optimal yet, we should be getting at least one over 0.5, which is not happening in any
for pair in tqdm(pairs):
    pairs[pair] = Jaccard_similarity(pair.split(","),MT)
    if pairs[pair] >= 0.5:
        print(pair)

100%|██████████| 11801/11801 [22:17<00:00,  8.82it/s]


In [108]:
# Max I've seen is 0.44
for pair in tqdm(pairs):
    if pairs[pair] >= 0.4:
        print(pair, pairs[pair])

100%|██████████| 11801/11801 [00:00<00:00, 3948387.17it/s]

69097,103046 0.44324667089410275
16179,56192 0.41611234294161126
10514,101679 0.40023134759976864
53912,88251 0.400399400898652
